## model training using Surprise module

In [46]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
import os
import io

from surprise import KNNBaseline, Reader
from surprise import Dataset

import cPickle as pickle
# load pickle of list id to list name
id_name_dic = pickle.load(open("popular_playlist.pkl","rb"))
#id_name_dic = pickle.load(open("playlist.pkl","rb"))
print("finish loading lists pickle...")
# rebuild the map dictionary for list
name_id_dic = {}
for playlist_id in id_name_dic:
    name_id_dic[id_name_dic[playlist_id]] = playlist_id
print("finish rebuilding dictionaries...")


file_path = os.path.expanduser('./popular_music_suprise_format.txt')
#file_path = os.path.expanduser('./163_music_suprise_format.txt')

# read data
reader = Reader(line_format='user item rating timestamp', sep=',')
music_data = Dataset.load_from_file(file_path, reader=reader)

print("build dataset...")
trainset = music_data.build_full_trainset()
#sim_options = {'name': 'pearson_baseline', 'user_based': False}

finish loading lists pickle...
finish rebuilding dictionaries...
build dataset...


In [47]:
id_name_dic.keys()[8]

'325440365'

In [48]:
print(id_name_dic[id_name_dic.keys()[8]])

有没有那么一首歌，会让你轻轻跟着和


In [49]:
trainset.n_items

50539

In [50]:
trainset.n_users

1076

#### 1.2.1 KNN -- basic collaborative filtering algorithm
find neighbors of the lists 

In [51]:
print("Begin Traning...")
#sim_options = {'user_based': False}
#algo = KNNBaseline(sim_options=sim_options)
algo = KNNBaseline()
algo.train(trainset)

current_playlist = name_id_dic.keys()[39]
print("list name", current_playlist)

# find out neighbors
# map list name to list id
playlist_id = name_id_dic[current_playlist]
print("listid", playlist_id)
# user id => to_inner_uid
playlist_inner_id = algo.trainset.to_inner_uid(playlist_id)
print("inner id", playlist_inner_id)

playlist_neighbors = algo.get_neighbors(playlist_inner_id, k=10)

# map list id to list name
# to_raw_uid
playlist_neighbors = (algo.trainset.to_raw_uid(inner_id)
                       for inner_id in playlist_neighbors)
playlist_neighbors = (id_name_dic[playlist_id]
                       for playlist_id in playlist_neighbors)

print()
print("the list: ", current_playlist, "》 is most similar with the following lists: \n")
for playlist in playlist_neighbors:
    print(playlist, algo.trainset.to_inner_uid(name_id_dic[playlist]))

Begin Traning...
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
list name 世事无常，唯愿你好
listid 306948578
inner id 427

the list:  世事无常，唯愿你好 》 is most similar with the following lists: 

【华语】暖心物语 纯白思念 3
暗暗作祟| 不甘朋友不敢恋人 15
专属你的周杰伦 18
「华语歌曲」 23
[小风收集]21世纪年轻人的音乐 24
十七岁那年，以为能和你永远 28
热门流行华语歌曲50首 31
最易上手吉他弹唱超精选 40
打开任意门，就有对的人 42
行车路上，一曲长歌 45


#### 1.2.2 songs recommendation (used-based prediction)

In [7]:
import cPickle as pickle
# reload songs pickle
song_id_name_dic = pickle.load(open("popular_song.pkl","rb"))
print("finish loadings songs pickle...")
# rebuild the songs map dictionary
song_name_id_dic = {}
for song_id in song_id_name_dic:
    song_name_id_dic[song_id_name_dic[song_id]] = song_id
print("finish building the map dictionaries...")

finish loadings songs pickle...
finish building the map dictionaries...


In [37]:
algo.predict?

In [8]:
# test the inner id
user_inner_id = 4
user_rating = trainset.ur[user_inner_id]
items = map(lambda x:x[0], user_rating)
for song in items:
    print(algo.predict(user_inner_id, song, r_ui=1), song_id_name_dic[algo.trainset.to_raw_iid(song)])

user: 4          item: 478        r_ui = 1.00   est = 1.00   {u'was_impossible': False} 听见下雨的声音	魏如昀
user: 4          item: 429        r_ui = 1.00   est = 1.00   {u'was_impossible': False} 梦一场	萧敬腾
user: 4          item: 936        r_ui = 1.00   est = 1.00   {u'was_impossible': False} 干杯	西瓜Kune
user: 4          item: 937        r_ui = 1.00   est = 1.00   {u'was_impossible': False} 给自己的歌 (Live) - live	纵贯线
user: 4          item: 938        r_ui = 1.00   est = 1.00   {u'was_impossible': False} 小半	陈粒
user: 4          item: 939        r_ui = 1.00   est = 1.00   {u'was_impossible': False} 思念是一种病(Live) - live	张震岳
user: 4          item: 940        r_ui = 1.00   est = 1.00   {u'was_impossible': False} 可以不可以	丁当
user: 4          item: 941        r_ui = 1.00   est = 1.00   {u'was_impossible': False} 秋酿	房东的猫
user: 4          item: 616        r_ui = 1.00   est = 1.00   {u'was_impossible': False} 退后	周杰伦
user: 4          item: 942        r_ui = 1.00   est = 1.00   {u'was_impossible': False} 阴天	莫文蔚
user:

### 2. SVD algorithm

In [14]:
# NMF: A collaborative filtering algorithm based on Non-negative Matrix Factorization.
from surprise import NMF, evaluate
from surprise import Dataset

file_path = os.path.expanduser('./popular_music_suprise_format.txt')
# read data
reader = Reader(line_format='user item rating timestamp', sep=',')
music_data = Dataset.load_from_file(file_path, reader=reader)

# build full dataset
algo = NMF()
trainset = music_data.build_full_trainset()
algo.train(trainset)

In [15]:
user_inner_id = 4
user_rating = trainset.ur[user_inner_id]
items = map(lambda x:x[0], user_rating)
for song in items:
    print(algo.predict(algo.trainset.to_raw_uid(user_inner_id), algo.trainset.to_raw_iid(song), r_ui=1), song_id_name_dic[algo.trainset.to_raw_iid(song)])

user: 400232387  item: 27724082   r_ui = 1.00   est = 1.00   {u'was_impossible': False} 听见下雨的声音	魏如昀
user: 400232387  item: 167916     r_ui = 1.00   est = 1.00   {u'was_impossible': False} 梦一场	萧敬腾
user: 400232387  item: 408307325  r_ui = 1.00   est = 1.00   {u'was_impossible': False} 干杯	西瓜Kune
user: 400232387  item: 394618     r_ui = 1.00   est = 1.00   {u'was_impossible': False} 给自己的歌 (Live) - live	纵贯线
user: 400232387  item: 421423806  r_ui = 1.00   est = 1.00   {u'was_impossible': False} 小半	陈粒
user: 400232387  item: 394485     r_ui = 1.00   est = 1.00   {u'was_impossible': False} 思念是一种病(Live) - live	张震岳
user: 400232387  item: 5239563    r_ui = 1.00   est = 1.00   {u'was_impossible': False} 可以不可以	丁当
user: 400232387  item: 30635613   r_ui = 1.00   est = 1.00   {u'was_impossible': False} 秋酿	房东的猫
user: 400232387  item: 185884     r_ui = 1.00   est = 1.00   {u'was_impossible': False} 退后	周杰伦
user: 400232387  item: 276936     r_ui = 1.00   est = 1.00   {u'was_impossible': False} 阴天	莫文蔚
user:

## Save the model

In [16]:
import surprise
surprise.dump.dump('./recommendation.model', algo=algo)
# reload by
#algo = surprise.dump.load('./recommendation.model')

## Comparison of different models

### load the data

In [17]:
import os
from surprise import Reader, Dataset
# specify the path
file_path = os.path.expanduser('./popular_music_suprise_format.txt')
# read data
reader = Reader(line_format='user item rating timestamp', sep=',')
music_data = Dataset.load_from_file(file_path, reader=reader)
# split the data into 5 folds
music_data.split(n_folds=5)

In [18]:
music_data.raw_ratings[:20]

[(u'715998294', u'375282', 1.0, u'1300000'),
 (u'36011068', u'185888', 1.0, u'1300000'),
 (u'53679351', u'355995', 1.0, u'1300000'),
 (u'8178769', u'375043', 1.0, u'1300000'),
 (u'79510350', u'254460', 1.0, u'1300000'),
 (u'63781839', u'27671258', 1.0, u'1300000'),
 (u'436300144', u'5282212', 1.0, u'1300000'),
 (u'101051505', u'32977064', 1.0, u'1300000'),
 (u'149264684', u'186816', 1.0, u'1300000'),
 (u'91208224', u'600190', 1.0, u'1300000'),
 (u'59394836', u'31654747', 1.0, u'1300000'),
 (u'58631872', u'374981', 1.0, u'1300000'),
 (u'389762547', u'307737', 1.0, u'1300000'),
 (u'405576826', u'421091657', 1.0, u'1300000'),
 (u'313037986', u'36664267', 1.0, u'1300000'),
 (u'389762547', u'33361858', 1.0, u'1300000'),
 (u'306435432', u'25641369', 1.0, u'1300000'),
 (u'324789304', u'326904', 1.0, u'1300000'),
 (u'87225065', u'28762176', 1.0, u'1300000'),
 (u'150974834', u'185994', 1.0, u'1300000')]

In [ ]:
### NormalPredictor
#from surprise import NormalPredictor, evaluate
#algo = NormalPredictor()
#perf = evaluate(algo, music_data, measures=['RMSE', 'MAE'])

In [20]:
### BaselineOnly
from surprise import BaselineOnly, evaluate
algo = BaselineOnly()
perf = evaluate(algo, music_data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm BaselineOnly.

------------
Fold 1
Estimating biases using als...
RMSE: 0.0000
MAE:  0.0000
------------
Fold 2
Estimating biases using als...
RMSE: 0.0000
MAE:  0.0000
------------
Fold 3
Estimating biases using als...
RMSE: 0.0000
MAE:  0.0000
------------
Fold 4
Estimating biases using als...
RMSE: 0.0000
MAE:  0.0000
------------
Fold 5
Estimating biases using als...
RMSE: 0.0000
MAE:  0.0000
------------
------------
Mean RMSE: 0.0000
Mean MAE : 0.0000
------------
------------


In [36]:
KNNBasic?

In [35]:
### KNNBasic
from surprise import KNNBasic, evaluate
algo = KNNBasic()
perf = evaluate(algo, music_data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm KNNBasic.

------------
Fold 1
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
Fold 2
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
Fold 3
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
Fold 4
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
Fold 5
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
------------
Mean RMSE: 0.0000
Mean MAE : 0.0000
------------
------------


In [22]:
### KNNWithMeans
from surprise import KNNWithMeans, evaluate
algo = KNNWithMeans()
perf = evaluate(algo, music_data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm KNNWithMeans.

------------
Fold 1
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
Fold 2
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
Fold 3
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
Fold 4
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
Fold 5
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
------------
Mean RMSE: 0.0000
Mean MAE : 0.0000
------------
------------


In [23]:
### KNNbaseline
from surprise import KNNBaseline, evaluate
algo = KNNBaseline()
perf = evaluate(algo, music_data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm KNNBaseline.

------------
Fold 1
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
Fold 2
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
Fold 3
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
Fold 4
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
Fold 5
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
------------
Mean RMSE: 0.0000
Mean MAE : 0.0000
------------
------------


In [24]:
### SVD
from surprise import SVD, evaluate
algo = SVD()
perf = evaluate(algo, music_data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.0362
MAE:  0.0166
------------
Fold 2
RMSE: 0.0369
MAE:  0.0167
------------
Fold 3
RMSE: 0.0367
MAE:  0.0167
------------
Fold 4
RMSE: 0.0374
MAE:  0.0172
------------
Fold 5
RMSE: 0.0365
MAE:  0.0167
------------
------------
Mean RMSE: 0.0367
Mean MAE : 0.0168
------------
------------


In [25]:
### SVD++
from surprise import SVDpp, evaluate
algo = SVDpp()
perf = evaluate(algo, music_data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm SVDpp.

------------
Fold 1
RMSE: 0.0150
MAE:  0.0065
------------
Fold 2
RMSE: 0.0149
MAE:  0.0064
------------
Fold 3
RMSE: 0.0152
MAE:  0.0066
------------
Fold 4
RMSE: 0.0154
MAE:  0.0066
------------
Fold 5
RMSE: 0.0152
MAE:  0.0065
------------
------------
Mean RMSE: 0.0151
Mean MAE : 0.0065
------------
------------


In [26]:
### NMF
from surprise import NMF
algo = NMF()
perf = evaluate(algo, music_data, measures=['RMSE', 'MAE'])
print_perf(perf)

Evaluating RMSE, MAE of algorithm NMF.

------------
Fold 1
RMSE: 0.0000
MAE:  0.0000
------------
Fold 2
RMSE: 0.0000
MAE:  0.0000
------------
Fold 3
RMSE: 0.0000
MAE:  0.0000
------------
Fold 4
RMSE: 0.0000
MAE:  0.0000
------------
Fold 5
RMSE: 0.0000
MAE:  0.0000
------------
------------
Mean RMSE: 0.0000
Mean MAE : 0.0000
------------
------------


NameError: name 'print_perf' is not defined